In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import torch
import pickle
from scipy.sparse import load_npz

import nbimporter
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import optuna
import wandb
import importlib
import ipynb
import torch.nn.functional as F

In [ ]:
from ipynb.fs.defs.recommenders_architecture import *
importlib.reload(ipynb.fs.defs.recommenders_architecture)
from ipynb.fs.defs.recommenders_architecture import *

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# Edit

Please note - the model needs to be edited in the Model initialization part inside the objective function

In [ ]:
saved_model_name = "NCF_with_Metadata_&_biases"

# Configurations

In [ ]:
current_dir = Path.cwd()
encoded_dir = current_dir.parent / "data" / "encoded"
user_item_dir = current_dir.parent / "data" / "pre_process"
save_dir = current_dir.parent / "models" 

encoded_titles_file = encoded_dir / "titles_encodings.pkl"
encoded_images_file = encoded_dir / "images_encodings.pkl"

In [ ]:
# with open(encoded_titles_file, 'rb') as f:
#     titles_embeddings = pickle.load(f)
# with open(encoded_images_file, 'rb') as f:
#     images_embeddings = pickle.load(f)

train = load_npz(user_item_dir / 'train.npz')
val = load_npz(user_item_dir / 'val.npz')

In [ ]:
class UserItemDataset(Dataset):
    def __init__(self, user_item_matrix, device):
        self.user_item_matrix = user_item_matrix
        self.indices = list(zip(*user_item_matrix.nonzero()))
        self.device = device

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        user_idx, item_idx = self.indices[idx]
        rating = self.user_item_matrix[user_idx, item_idx]

        return (
            torch.tensor(user_idx, dtype=torch.long),
            torch.tensor(item_idx, dtype=torch.long),
            torch.tensor(rating, dtype=torch.float))

# Training functions

In [ ]:
def early_stop_check(patience, best_val_loss, best_val_loss_epoch, current_val_loss, current_val_loss_epoch):
    early_stop_flag = False 
    if current_val_loss < best_val_loss:
        best_val_loss = current_val_loss
        best_val_loss_epoch = current_val_loss_epoch
    else:
        if current_val_loss_epoch - best_val_loss_epoch > patience:
            early_stop_flag = True  # Change flag
    return best_val_loss, best_val_loss_epoch, early_stop_flag

In [ ]:
def train_model_with_hyperparams(model, train_loader, val_loader, criterion, optimizer, patience, epochs, device):
    best_val_loss = float('inf')
    best_val_loss_epoch = 0
    early_stop_flag = False

    for epoch in range(epochs):
        if early_stop_flag:
            break

        model.train()
        train_loss = 0.0
        total_samples_train = 0
        for user_idx, item_idx, ratings in train_loader:
            user_idx, item_idx, ratings = user_idx.to(device), item_idx.to(device), ratings.to(device)

            optimizer.zero_grad()
            predictions = model(user_idx, item_idx)
            loss = criterion(predictions, ratings)

            loss.backward()
            optimizer.step()
            
            batch_size = ratings.size(0)
            train_loss += loss.item() * batch_size 
            total_samples_train += batch_size 
            
        train_loss /= total_samples_train

        model.eval()
        val_loss = 0.0
        total_samples_val = 0
        with torch.no_grad():
            for user_idx, item_idx, ratings in val_loader:
                user_idx, item_idx, ratings = user_idx.to(device), item_idx.to(device), ratings.to(device)

                predictions = model(user_idx, item_idx)
                loss = criterion(predictions, ratings)
                
                batch_size = ratings.size(0)
                val_loss += loss.item() * batch_size 
                total_samples_val += batch_size 

        val_loss /= total_samples_val

        if val_loss < best_val_loss:    
            model_name = f"{saved_model_name}_{epoch}_{val_loss}.pth"      
            model_path = save_dir / model_name
            torch.save(model, model_path)
            
        # Early stopping check
        best_val_loss, best_val_loss_epoch, early_stop_flag = early_stop_check(patience, best_val_loss, best_val_loss_epoch, val_loss, epoch)

        wandb.log({"epoch": epoch + 1, "train_loss": train_loss, "val_loss": val_loss})

    return best_val_loss

In [ ]:
def objective(trial):
    # Hyperparameters to tune
    embedding_dim = trial.suggest_categorical("embedding_dim", [8, 16, 24, 32, 40])
    hidden_units = trial.suggest_categorical("hidden_units", [[32, 16], [64, 32], [128, 64], [256, 128]])
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    lr = trial.suggest_categorical("learning_rate", [1e-6, 1e-5, 1e-4, 1e-3, 1e-2])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [1024, 2048, 4096, 8192])
    alpha = trial.suggest_float("alpha", 0.0, 1.0)
    epochs = 30

    # W&B initialization
    wandb.init(project=f"RS_project_{saved_model_name}", 
               name=f"trial-{trial.number}", 
               config={"embedding_dim": embedding_dim,
                       "hidden_units": hidden_units,
                       "dropout": dropout,
                       "batch_size": batch_size,
                       "learning_rate": lr,
                       "weight_decay": weight_decay,
                       "alpha": alpha})
    
    # Data preparation
    train_dataset = UserItemDataset(train, device)
    val_dataset = UserItemDataset(val, device)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    
    # Load bias data
    current_dir = Path.cwd()
    user_item_file_path = current_dir.parent / "data" / "data_and_test_files" / "user_item_rating_table_train_with_idx.csv"
    df2 = pd.read_csv(user_item_file_path)

    compressed_items_encodings = encoded_dir / "compressed_all_data_encodings_256.pkl"
    with open(compressed_items_encodings, 'rb') as f:
        compressed_items_encodings = pickle.load(f)
    
    # Model initialization
    model = NCFWithMetadata(num_users=train.shape[0], num_items=train.shape[1], embedding_dim=embedding_dim, hidden_units=hidden_units, dropout=dropout, alpha=alpha, df2=df2, compressed_items_encodings=compressed_items_encodings, device=device).to(device)
    
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # Train and validate
    best_val_loss = train_model_with_hyperparams(model, train_loader, val_loader, criterion, optimizer, patience=10, epochs=epochs, device=device)
    
    wandb.finish()
    return best_val_loss

# Create Optuna study using the functions

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Best hyperparameters:", study.best_params)